# Advanced NLP Models for Question-Answering

In this notebook, you'll explore some of the more advanced Transformer models for Question-Answering - BERT, T5, ALBERT, ELECTRA, and Longformer. This workshop provides an in-depth look into their architecture and applications. The subdomain of Question Answering in NLP is popular because of the wide range of applications in answering questions with a reference document. The solution to this problem is tackled by using a dataset that consists of an input text, a query, and the segment of text or span from the input text which contains the answer to the query. With the help of deep learning models, there has been a significant improvement in achieving human-level predictions from the data.

#### Installation Guide

In [ ]:
#!pip install transformers torch

### The Transformer Architecture
The transformer architecture was proposed in the paper Attention is All You Need. The encoders encode the input text, and the decoder processes the encodings to understand the contextual information behind the sequence. Each encoder and decoder in the stack uses an attention mechanism to process each input along with every other input for weighing their relevance, and generates the output sequence with the help of the decoder. The attention mechanism enables dynamic highlighting and understanding the features in the input text.

<img src="https://lh3.googleusercontent.com/d/1kMk1ZgOi5FA2GpTODXCWjdXBfmfSr9A_" alt="drawing" height="450">

### BERT (Bidirectional Encoder Representations from Transformers)

The huggingface `transformers` library provides nicely wrapped implementations for us to experiment with. 



Let's begin our experiments by first diving into a BERT model for understanding its performance. BERT (Bidirectional Encoder Representations from Transformers) is a pre-trained NLP model developed by Google that uses bidirectional attention to understand the context of words in a sentence. It can be fine-tuned for various tasks, including text classification, question answering, and language generation.

<img src="https://lh3.googleusercontent.com/d/1ye4IvOzGM4b2ix1e2ByMntCUv1rM9G7n" alt="drawing" width="650"><br><br>
<img src="https://lh3.googleusercontent.com/d/15D_t6bQ3ziz0b2pUIbGHxGTKlJSsbWe0" alt="drawing" width="450">
<img src="https://lh3.googleusercontent.com/d/1e5UhuKT5v1Jn0nsqNmEeIA9SPQMGOgeH" alt="drawing" width="350">

In [1]:
from transformers import BertTokenizer,AlbertTokenizer,AutoTokenizer, AutoModelForQuestionAnswering ,BertForQuestionAnswering, AlbertForQuestionAnswering
import torch

model_name='bert-large-uncased-whole-word-masking-finetuned-squad'
model = BertForQuestionAnswering.from_pretrained(model_name) 

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
tokenizer = BertTokenizer.from_pretrained(model_name)
question = "Who discovered penicillin?"
answer_text = "Penicillin was discovered in 1928 by Scottish scientist Sir Alexander Fleming. His discovery led to the development of antibiotics, which have saved countless lives."

input_ids = tokenizer.encode(question, answer_text)
tokens = tokenizer.convert_ids_to_tokens(input_ids)
print(tokens)

['[CLS]', 'who', 'discovered', 'pen', '##ici', '##llin', '?', '[SEP]', 'pen', '##ici', '##llin', 'was', 'discovered', 'in', '1928', 'by', 'scottish', 'scientist', 'sir', 'alexander', 'fleming', '.', 'his', 'discovery', 'led', 'to', 'the', 'development', 'of', 'antibiotics', ',', 'which', 'have', 'saved', 'countless', 'lives', '.', '[SEP]']


In [3]:
def qa(question,answer_text,model,tokenizer):
  inputs = tokenizer.encode_plus(question, answer_text, add_special_tokens=True, return_tensors="pt")
  input_ids = inputs["input_ids"].tolist()[0]

  text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
  print(text_tokens)
  outputs = model(**inputs)
  answer_start_scores=outputs.start_logits
  answer_end_scores=outputs.end_logits

  answer_start = torch.argmax(
      answer_start_scores
  )  # Get the most likely beginning of answer with the argmax of the score
  answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score
  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
  
  # Combine the tokens in the answer and print it out.""
  answer = answer.replace("#","")

  print('Answer: "' + answer + '"')
  return answer

In [4]:
qa(question,answer_text,model,tokenizer)

['[CLS]', 'who', 'discovered', 'pen', '##ici', '##llin', '?', '[SEP]', 'pen', '##ici', '##llin', 'was', 'discovered', 'in', '1928', 'by', 'scottish', 'scientist', 'sir', 'alexander', 'fleming', '.', 'his', 'discovery', 'led', 'to', 'the', 'development', 'of', 'antibiotics', ',', 'which', 'have', 'saved', 'countless', 'lives', '.', '[SEP]']
Answer: "sir alexander fleming"


'sir alexander fleming'

### Advanced Transformer Models

#### T5

T5, developed by Google Research, frames every NLP task as a text-to-text problem, providing a unified framework for tasks like translation, summarization, and question answering. It excels at these tasks by leveraging large-scale pretraining on diverse text data with a denoising objective.

In [12]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def t5_qa(question, context, model, tokenizer):
    input_text = f"question: {question} context: {context}"
    inputs = tokenizer.encode_plus(input_text, return_tensors="pt", add_special_tokens=True)
    
    outputs = model.generate(inputs['input_ids'])
    answer = tokenizer.decode(outputs[0])
    
    print('Answer:', answer)

t5_qa(question, answer_text, model, tokenizer)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Answer: <pad> Sir Alexander Fleming</s>


#### ALBERT (A Lite BERT)
ALBERT, developed by Google Research, is a smaller, faster BERT variant with significantly reduced memory requirements. It achieves comparable performance to BERT by using parameter-sharing techniques and sentence-order prediction, making it ideal for scaling NLP applications.

In [6]:
tokenizer=AlbertTokenizer.from_pretrained('ahotrod/albert_xxlargev1_squad2_512')
model=AlbertForQuestionAnswering.from_pretrained('ahotrod/albert_xxlargev1_squad2_512')
qa(question,answer_text,model,tokenizer)

Some weights of the model checkpoint at ahotrod/albert_xxlargev1_squad2_512 were not used when initializing AlbertForQuestionAnswering: ['albert.pooler.bias', 'albert.pooler.weight']
- This IS expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['[CLS]', '▁who', '▁discovered', '▁pen', 'ici', 'll', 'in', '?', '[SEP]', '▁pen', 'ici', 'll', 'in', '▁was', '▁discovered', '▁in', '▁1928', '▁by', '▁scottish', '▁scientist', '▁sir', '▁alexander', '▁fleming', '.', '▁his', '▁discovery', '▁led', '▁to', '▁the', '▁development', '▁of', '▁antibiotic', 's', ',', '▁which', '▁have', '▁saved', '▁countless', '▁lives', '.', '[SEP]']
Answer: "sir alexander fleming"


'sir alexander fleming'

#### ELECTRA (Efficiently Learning an Encoder that Classifies Token Replacements Accurately): 
ELECTRA, also by Google Research, replaces BERT's masked language modeling with a "discriminator-generator" approach. The generator corrupts input tokens, and the discriminator identifies altered tokens, training models efficiently to provide high-quality language representations.

In [8]:
tokenizer = AutoTokenizer.from_pretrained("valhalla/electra-base-discriminator-finetuned_squadv1")
model = AutoModelForQuestionAnswering.from_pretrained("valhalla/electra-base-discriminator-finetuned_squadv1")

qa(question,answer_text,model,tokenizer)

['[CLS]', 'who', 'discovered', 'pen', '##ici', '##llin', '?', '[SEP]', 'pen', '##ici', '##llin', 'was', 'discovered', 'in', '1928', 'by', 'scottish', 'scientist', 'sir', 'alexander', 'fleming', '.', 'his', 'discovery', 'led', 'to', 'the', 'development', 'of', 'antibiotics', ',', 'which', 'have', 'saved', 'countless', 'lives', '.', '[SEP]']
Answer: "sir alexander fleming"


'sir alexander fleming'

### Long Document Question-Answering Using Standard Models
 The transformers model uses the help of the self-attention operation to provide meaningful results. As the length of the sequence increases, the computation scales drastically for this mechanism. If we use the standard BERT model as we used earlier, an error will be observed, indicating that the long sequence of inputs cannot be processed.

In [ ]:
question="how many pilots did the ship have?"

answer_text="""
Tug boats had spent several hours on Monday working to free the bow of the massive vessel after dislodging the stern earlier in the day.
Marine traffic websites showed images of the ship away from the banks of the Suez Canal for the first time in seven days following an around-the-clock international effort to reopen the global shipping lane.
There are still 422 ships waiting to go through the Suez Canal, Rabie said, adding that the canal's authorities decided the ships will be able to cross the canal on a first come first serve basis, though the ships carrying livestock were permitted to cross in the first convoy of the day.
The average number of ships that transited through the canal on a daily basis before the accident was between 80 to 90 ships, according to Lloyds List; however, the head of the Suez Canal Authority said that the channel will work over 24 hours a day to facilitate the passage of almost 400 ships carrying billions of dollars in freight.
The journey to cross the canal takes 10 to 12 hours and in the event the channel operates for 24 hours, two convoys per day will be able to successfully pass through.
Still, shipping giant Maersk issued an advisory telling customers it could take "6 days or more" for the line to clear. The company said that was an estimate and subject to change as more vessels reach the blockage or are diverted.
The rescue operation had intensified in both urgency and global attention with each day that passed, as ships from around the world, carrying vital fuel and cargo, were blocked from entering the canal during the crisis, raising alarm over the impact on global supply chains.
What its really like steering the worlds biggest ships
What it's really like steering the world's biggest ships
Promising signs first emerged earlier on Monday when the rear of the vessel was freed from one of the canal's banks.
People at the canal cheered as news of Monday's progress came in.
The Panama Maritime Authority said that according to preliminary reports Ever Given suffered mechanical problems that affected its maneuverability.

The ship had two pilots on board during the transit.

However, the owner of the vessel, Japanese shipping company Shoe Kisen insists that there had been no blackout resulting in loss of power prior to the ship’s grounding.
Instead, gusting winds of 30 knots and poor visibility due to a dust storm have been identified as the likely causes of the grounding, which left the boxship stuck sideways in a narrow point of the waterway.
"""

from transformers import BertTokenizer,BertForQuestionAnswering
model_name='bert-large-uncased-whole-word-masking-finetuned-squad'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

qa(question,answer_text,model,tokenizer)

We need another model which has been pre-trained to include a longer sequence of input documents, and also an architecture that can process the same.

#### Longformer
Developed by the Allen Institute for AI, Longformer is a Transformer model designed to handle long documents. It uses a combination of local and global attention mechanisms to reduce memory requirements, allowing it to capture relationships in sequences of up to 4,096 tokens while maintaining efficient computational performance. Longformer is particularly suited for tasks involving lengthy documents like text classification, summarization, and question answering.

In [15]:
tokenizer = AutoTokenizer.from_pretrained("valhalla/longformer-base-4096-finetuned-squadv1")

model = AutoModelForQuestionAnswering.from_pretrained("valhalla/longformer-base-4096-finetuned-squadv1")
qa(question,answer_text,model,tokenizer)

Some weights of the model checkpoint at valhalla/longformer-base-4096-finetuned-squadv1 were not used when initializing LongformerForQuestionAnswering: ['longformer.pooler.dense.bias', 'longformer.pooler.dense.weight']
- This IS expected if you are initializing LongformerForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Input ids are automatically padded from 562 to 1024 to be a multiple of `config.attention_window`: 512


['<s>', 'how', 'Ġmany', 'Ġpilots', 'Ġdid', 'Ġthe', 'Ġship', 'Ġhave', '?', '</s>', '</s>', 'Ċ', 'T', 'ug', 'Ġboats', 'Ġhad', 'Ġspent', 'Ġseveral', 'Ġhours', 'Ġon', 'ĠMonday', 'Ġworking', 'Ġto', 'Ġfree', 'Ġthe', 'Ġbow', 'Ġof', 'Ġthe', 'Ġmassive', 'Ġvessel', 'Ġafter', 'Ġdisl', 'od', 'ging', 'Ġthe', 'Ġstern', 'Ġearlier', 'Ġin', 'Ġthe', 'Ġday', '.', 'Ċ', 'Mar', 'ine', 'Ġtraffic', 'Ġwebsites', 'Ġshowed', 'Ġimages', 'Ġof', 'Ġthe', 'Ġship', 'Ġaway', 'Ġfrom', 'Ġthe', 'Ġbanks', 'Ġof', 'Ġthe', 'ĠS', 'uez', 'ĠCanal', 'Ġfor', 'Ġthe', 'Ġfirst', 'Ġtime', 'Ġin', 'Ġseven', 'Ġdays', 'Ġfollowing', 'Ġan', 'Ġaround', '-', 'the', '-', 'clock', 'Ġinternational', 'Ġeffort', 'Ġto', 'Ġreopen', 'Ġthe', 'Ġglobal', 'Ġshipping', 'Ġlane', '.', 'Ċ', 'There', 'Ġare', 'Ġstill', 'Ġ422', 'Ġships', 'Ġwaiting', 'Ġto', 'Ġgo', 'Ġthrough', 'Ġthe', 'ĠS', 'uez', 'ĠCanal', ',', 'ĠRab', 'ie', 'Ġsaid', ',', 'Ġadding', 'Ġthat', 'Ġthe', 'Ġcanal', "'s", 'Ġauthorities', 'Ġdecided', 'Ġthe', 'Ġships', 'Ġwill', 'Ġbe', 'Ġable', 'Ġto', 'Ġc

' two'

### Conclusion

In this notebook, we have learnt to apply various transformer-based models for question answering. T5's text-to-text approach, ALBERT's efficient parameter-sharing, ELECTRA's novel discriminator-generator learning, and Longformer's scalability are all vital contributions to modern NLP. As you continue experimenting and applying what you've learned, you'll find innovative ways to leverage these models in your projects.